In [2]:
import requests
import re
import glob
from bs4 import BeautifulSoup
import pandas as pd
import time


df = pd.read_csv('/home/hhs/pdbwork/pdb_work_list.csv')

uni_list =df['unipro_ac']

sort_uni_list = list(sorted(set(uni_list)))

sec = 0.5
def replace_sig(strings):
    if '%28' in strings:
        strings = strings.replace('%28', '(')
    if '%29' in strings:
        strings = strings.replace('%29', ')')
    if '%2F' in strings:
        strings = strings.replace('%2F', '/')
    if '+' in strings:
        strings = strings.replace('+', '_')
    if '%5B' in strings:
        strings = strings.replace('%5B', '[')
    if '%5D' in strings:
        strings = strings.replace('%5D', ']')
    if '%3A' in strings:
        strings = strings.replace('%3A', ':')
    if '%27' in strings:
        strings = strings.replace('%27', '^')
    if '%2C' in strings:
        strings = strings.replace('%2C', ',')
        
    return strings


sufam_list = []
fam_list =[]
subfam_list = []
for i in range(2299,len(sort_uni_list)):
#for i in [42]:
    #if i%100 == 99:
    time.sleep(sec)
    #    print('DDos오해를 풀기위해 1분간 쉬는중...')
    #del ret, ret_2, ret2, ret3, stret3, lstret3, ret4, ret5
    ret = []
    ret_2 = []
    ret3 = []
    stret3 = []
    lstret3 = []
    ret4 = []
    ret5 = []
    unipro_id = sort_uni_list[i]
    aaa = 'http://www.uniprot.org/uniprot/' + unipro_id
    bbb = requests.get(aaa)

    soup = BeautifulSoup(bbb.text)

    ret =soup.find("div",{'class':'section', 'id': 'family_and_domains'})
    if not ret:
        print('no.{},{} dont hav ret1'.format(i,unipro_id))
        supfam_name = 'no_supfam'
        fam_name = 'no_fam'
        subfam_name = 'no_subfam'
        
        print(supfam_name, fam_name, subfam_name)
                    
        sufam_list.append(supfam_name)
        fam_list.append(fam_name)
        subfam_list.append(subfam_name)
        continue
    ret_2 = ret.find_all("div", {'class': 'annotation', 'property':'hasPart' , 'typeof':'CreativeWork'})
    #ret3 = ret2.find_all('a')[1]
    #ret2 = [i for i in rettt if 'Belongs' in str(i)][0]
   
    if ret_2:
        sampling2 = [i for i in ret_2 if 'elongs to' in str(i)] 
        if not sampling2:
            print('no.{},{} hav no family.'.format(i,unipro_id))
            ret3 =[]
        else:
            ret2 = sampling2[0]
            ret3 = ret2.find_all('a')
            print('no.{},{} has len ret2: {}'.format(i,unipro_id, len(ret2)))
        #ret3 = ret2.find_all('a')
        stret3 = [str(i) for i in ret3]
        lstret3 = [i for i in stret3 if 'query=family' in i]
        
        if len(lstret3) >0:
            
            print('no.{},{} has lstret3:{}'.format(i,unipro_id,lstret3[-1]))
            ret4 = str(lstret3[-1])
            ret5 = ret4.split('%22')[1]
            check_len = len(ret5.split('.+'))
            if check_len ==3:
                supf_name = ret5.split('.+')[0]
                family_name =ret5.split('.+')[1]
                subf_name = ret5.split('.+')[2]
                
                supfam_name = replace_sig(supf_name) 
                fam_name = replace_sig(family_name)
                subfam_name = replace_sig(subf_name)

                print(supfam_name, fam_name, subfam_name)
                
                sufam_list.append(supfam_name)
                fam_list.append(fam_name)
                subfam_list.append(subfam_name)
            
            elif check_len ==2:
                if 'superfamily' in ret5:
                    supf_name =  ret5.split('.+')[0]
                    family_name =ret5.split('.+')[1]
                    
                    supfam_name = replace_sig(supf_name) 
                    fam_name = replace_sig(family_name)
                    subfam_name = 'no_subfam'
                    print(supfam_name, fam_name, subfam_name)
                    
                    sufam_list.append(supfam_name)
                    fam_list.append(fam_name)
                    subfam_list.append(subfam_name)
                else:
                    family_name = ret5.split('.+')[0]
                    subf_name = ret5.split('.+')[1]
                    
                    supfam_name = 'no_supfam'
                    fam_name = replace_sig(family_name)
                    subfam_name = replace_sig(subf_name)
                    print(supfam_name, fam_name, subfam_name)
                    
                    sufam_list.append(supfam_name)
                    fam_list.append(fam_name)
                    subfam_list.append(subfam_name)
                    
            elif check_len ==1:
                
                supfam_name = 'no_supfam'
                fam_name = replace_sig(ret5)
                subfam_name = 'no_subfam'

                #print(supfam_name, fam_name, subfam_name)
                
                if 'superfamily' in ret:
                    sufam_list.append(fam_name)
                    fam_list.append('no_fam')
                    subfam_list.append(subfam_name)
                    print( fam_name,'no_fam', subfam_name)
                else:
                    sufam_list.append(supfam_name)
                    fam_list.append(fam_name)
                    subfam_list.append(subfam_name)
                    print(supfam_name, fam_name, subfam_name)
            else:
                print('no.{} not computed:{}'.format(i,unipro_id))
        
        
        else: #lstret3의 길이가 0일경우
            print('no.{},{} dont hav lstret3... ret2:{}'.format(i,unipro_id,ret2))
            
            #ret4 = str(lstret3[0])
            supfam_name = 'no_supfam'
            fam_name = 'no_fam'
            subfam_name = 'no_subfam'
            print(supfam_name, fam_name, subfam_name)
                    
            sufam_list.append(supfam_name)
            fam_list.append(fam_name)
            subfam_list.append(subfam_name)
            
    else:
        print('no.{},{} dont hav ret_2'.format(i,unipro_id))
        supfam_name = 'no_supfam'
        fam_name = 'no_fam'
        subfam_name = 'no_subfam'
        
        print(supfam_name, fam_name, subfam_name)
                    
        sufam_list.append(supfam_name)
        fam_list.append(fam_name)
        subfam_list.append(subfam_name)

no.2299,Q81JT5 dont hav ret_2
no_supfam no_fam no_subfam
no.2300,Q81R22 has len ret2: 4
no.2300,Q81R22 has lstret3:<a href="/uniprot/?query=family:%22dihydrofolate+reductase+family%22&amp;sort=score">dihydrofolate reductase family</a>
no_supfam dihydrofolate_reductase_family no_subfam
no.2301,Q81RP3 has len ret2: 4
no.2301,Q81RP3 has lstret3:<a href="/uniprot/?query=family:%22NAD+synthetase+family%22&amp;sort=score">NAD synthetase family</a>
no_supfam NAD_synthetase_family no_subfam
no.2302,Q81RQ8 dont hav ret_2
no_supfam no_fam no_subfam
no.2303,Q81S73 has len ret2: 5
no.2303,Q81S73 has lstret3:<a href="/uniprot/?query=family:%22UMP+kinase+family%22&amp;sort=score">UMP kinase family</a>
no_supfam UMP_kinase_family no_subfam
no.2304,Q81TP1 has len ret2: 4
no.2304,Q81TP1 has lstret3:<a href="/uniprot/?query=family:%22dTDP-4-dehydrorhamnose+3%2C5-epimerase+family%22&amp;sort=score">dTDP-4-dehydrorhamnose 3,5-epimerase family</a>
no_supfam dTDP-4-dehydrorhamnose_3,5-epimerase_family no_su

In [3]:
print('sufam:{}, fam:{}, subfam:{}'.format(len(sufam_list), len(fam_list), len(subfam_list)))

sufam:594, fam:594, subfam:594


In [4]:
start_num = 2299
data_length = len(sufam_list)

sort_unis = sort_uni_list[start_num:start_num+data_length]
dfs = pd.DataFrame({'uniprot_id': sort_unis, 'superfamily': sufam_list, 'family':fam_list, 'subfamily': subfam_list})

In [6]:
dfs.to_csv('/home/hhs/pdbwork/pdb_family2.csv')

In [7]:
sort_uni_list[2298:2300]

['Q81JJ9', 'Q81JT5']